In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import pandas as pd
import numpy as np
import datetime
import time 

In [5]:
options = Options() 
options.add_experimental_option("excludeSwitches", ['enable-automation']) # allows us to scrape bloomberg
#options.add_argument("--disable-notifications") #just added
options.add_argument("--start-maximized")

In [6]:
def fix_ticker_names(ticker):
    if "|" not in ticker:
        return ticker.upper()
    else:
        return ticker.split("|",1)[1].strip().upper()
    
def get_historical_data(ticker):
    ticker_data = "https://finance.yahoo.com/quote/%s/history?p=%s&.tsrc=fin-srch"%(ticker,ticker)
    
    browser = webdriver.Chrome(chrome_options=options, executable_path=r"/usr/bin/chromedriver")
    browser.get(ticker_data)
    time.sleep(3)
    browser.find_element_by_css_selector('a[class*= "Fl(end) Mt(3px) Cur(p)"]').click()
    time.sleep(5)
    browser.quit()
    return
    
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [7]:
news_scraping_results = pd.read_csv('./news_scraping_results.csv')
ticker_names = news_scraping_results["SYMBOL"].apply(lambda x: fix_ticker_names(x))
unique_tickers = np.unique(news_scraping_results["SYMBOL"].apply(lambda x: fix_ticker_names(x)))

In [21]:
# grab the historical ticker data from yahoo finance 
for ticker in unique_tickers: 
    get_historical_data(ticker)

/home/albert/anaconda3/envs/cs/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


WebDriverException: Message: chrome not reachable
  (Session info: chrome=77.0.3865.90)
  (Driver info: chromedriver=2.41.578700 (2f1ed5f9343c13f73144538f15c00b370eda6706),platform=Linux 5.0.0-31-generic x86_64)


In [8]:
import glob
import os
import re 

path = r'/home/albert/CS-SENTIMENT-ANALYSIS/ticker_historical_data/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    ticker_name = re.search('./ticker_historical_data/(.*).csv', filename)
    df.insert(0, column='SYMBOL', value=ticker_name.group(1))
    li.append(df)

ticker_historical_data = pd.concat(li, axis=0, ignore_index=True)
ticker_historical_data['Date'] = pd.to_datetime(ticker_historical_data['Date'], errors='coerce')
ticker_historical_data['Date'] = ticker_historical_data['Date'].apply(lambda x: x.date())

In [9]:
news_scraping_results['SYMBOL'] = news_scraping_results.SYMBOL.apply(lambda x: fix_ticker_names(x))
news_scraping_results['PUBLISH_TIME'] = pd.to_datetime(news_scraping_results['PUBLISH_TIME'], errors='coerce')

In [369]:
forecast_df = pd.DataFrame(columns=np.concatenate((['SYMBOL', 'URL'], np.arange(11))))

for index, row in news_scraping_results.iterrows():
    
    specific_ticker = ticker_historical_data[ticker_historical_data['SYMBOL'] == row['SYMBOL']]
    specific_ticker = specific_ticker.reset_index(drop=True)
    
    # check to see if publish date is present in stock dates
    if row['PUBLISH_TIME'].date() not in specific_ticker['Date']:
        # check to see if article publish date is within 3 days of an existing stock price 
        # or just grab the closest close stock price date for ticker 
        diff = (specific_ticker['Date'] - row['PUBLISH_TIME'].date())
        
        if diff[(diff < pd.to_timedelta(0))].empty: 
            pseudo_forecast = pd.DataFrame([[row['SYMBOL'],
                                         row['URL']]],
                                   columns=['SYMBOL', 'URL'])
            forecast_df = forecast_df.append(pseudo_forecast, ignore_index=True, sort=False)
        else: 
            indexmax = (diff[(diff < pd.to_timedelta(0))].idxmax())
            specific_ticker.iloc[[indexmax]]

            specific_range = specific_ticker[indexmax-10:indexmax+1]
            specific_range = specific_range.reset_index(drop=True)
            if len(specific_range) < 10: 
                pseudo_forecast = pd.DataFrame([[row['SYMBOL'],
                                             row['URL']]],
                                       columns=['SYMBOL', 'URL'])
                forecast_df = forecast_df.append(pseudo_forecast, ignore_index=True, sort=False)

            else: 
                pseudo_forecast = pd.DataFrame([np.concatenate(([row['SYMBOL'],row['URL']], specific_range['Close'][:11]))],
                                               columns=np.concatenate((['SYMBOL', 'URL'], np.arange(11))))

                forecast_df = forecast_df.append(pseudo_forecast, ignore_index=True, sort=False)

    else: 
        article_date_index = specific_ticker[specific_ticker['Date']==row['PUBLISH_TIME'].date()].index[0]
        specific_range = specific_ticker[article_date_index-10:article_date_index+1]
        specific_range = specific_range.reset_index(drop=True)
        
        if len(specific_range) < 11: 
            pseudo_forecast = pd.DataFrame([[row['SYMBOL'],
                                         row['URL']]],
                                   columns=['SYMBOL', 'URL'])
            forecast_df = forecast_df.append(pseudo_forecast, ignore_index=True, sort=False)
        
        else: 
            pseudo_forecast = pd.DataFrame([np.concatenate(([row['SYMBOL'],row['URL']], specific_range['Close'][:11]))],
                                           columns=np.concatenate((['SYMBOL', 'URL'], np.arange(11))))

            forecast_df = forecast_df.append(pseudo_forecast, ignore_index=True, sort=False)


In [385]:
forecast_df[['0','1','2','3','4','5','6','7','8','9','10']] = forecast_df[['0','1','2','3','4','5','6','7','8','9','10']].astype(np.float)

In [387]:
forecast_df['average'] = forecast_df[['0','1','2','3','4','5','6','7','8','9']].mean(axis=1)
forecast_df

,SYMBOL,URL,0,1,2,3,4,5,6,7,8,9,10,average
0,NKE,http://onlinenewsguru.com/2019/09/01/investors...,80.279999,81.129997,80.529999,82.739998,83.309998,80.440002,82.250000,82.029999,83.480003,85.379997,84.500000,82.156999
1,C,http://www.bnnbloomberg.ca/cp-rail-raises-quar...,69.669998,69.089996,69.199997,68.830002,68.730003,69.510002,71.029999,70.699997,69.959999,70.360001,70.669998,69.707999
2,UFPT,http://www.bnnbloomberg.ca/unilever-subsidiary...,35.270000,35.490002,35.740002,36.099998,37.400002,37.400002,36.150002,36.939999,36.810001,36.349998,36.240002,36.365001
3,TER,http://www.businesskorea.co.kr/news/articleVie...,30.780001,31.900000,32.169998,32.330002,31.490000,31.790001,31.709999,32.110001,32.900002,31.799999,31.930000,31.898000
4,EMMAW,http://www.globenewswire.com/news-release/2019...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,BYND,https://www.usatoday.com/story/money/2019/10/0...,142.990005,138.320007,154.339996,151.660004,148.619995,146.419998,143.300003,145.440002,145.720001,145.059998,142.729996,146.187001
2570,MSFT,https://www.wsj.com/articles/bright-horizons-t...,120.220001,117.050003,117.660004,117.910004,116.769997,116.930000,117.940002,119.019997,119.190002,119.970001,119.360001,118.266001
2571,AHH,https://www.wtrf.com/local/main-street-bank-an...,16.750000,16.700001,16.799999,16.790001,16.840000,16.940001,17.059999,17.020000,17.139999,17.240000,17.150000,16.928000
2572,TER,https://www.yahoo.com/entertainment/hgtvs-wind...,46.230000,45.450001,45.939999,46.650002,47.910000,48.830002,47.549999,47.299999,46.910000,46.650002,46.349998,46.942000


In [388]:
forecast_df['diff'] = forecast_df['10'] - forecast_df['average']
forecast_df

,SYMBOL,URL,0,1,2,3,4,5,6,7,8,9,10,average,diff
0,NKE,http://onlinenewsguru.com/2019/09/01/investors...,80.279999,81.129997,80.529999,82.739998,83.309998,80.440002,82.250000,82.029999,83.480003,85.379997,84.500000,82.156999,2.343001
1,C,http://www.bnnbloomberg.ca/cp-rail-raises-quar...,69.669998,69.089996,69.199997,68.830002,68.730003,69.510002,71.029999,70.699997,69.959999,70.360001,70.669998,69.707999,0.961999
2,UFPT,http://www.bnnbloomberg.ca/unilever-subsidiary...,35.270000,35.490002,35.740002,36.099998,37.400002,37.400002,36.150002,36.939999,36.810001,36.349998,36.240002,36.365001,-0.124999
3,TER,http://www.businesskorea.co.kr/news/articleVie...,30.780001,31.900000,32.169998,32.330002,31.490000,31.790001,31.709999,32.110001,32.900002,31.799999,31.930000,31.898000,0.032000
4,EMMAW,http://www.globenewswire.com/news-release/2019...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,BYND,https://www.usatoday.com/story/money/2019/10/0...,142.990005,138.320007,154.339996,151.660004,148.619995,146.419998,143.300003,145.440002,145.720001,145.059998,142.729996,146.187001,-3.457005
2570,MSFT,https://www.wsj.com/articles/bright-horizons-t...,120.220001,117.050003,117.660004,117.910004,116.769997,116.930000,117.940002,119.019997,119.190002,119.970001,119.360001,118.266001,1.094000
2571,AHH,https://www.wtrf.com/local/main-street-bank-an...,16.750000,16.700001,16.799999,16.790001,16.840000,16.940001,17.059999,17.020000,17.139999,17.240000,17.150000,16.928000,0.222000
2572,TER,https://www.yahoo.com/entertainment/hgtvs-wind...,46.230000,45.450001,45.939999,46.650002,47.910000,48.830002,47.549999,47.299999,46.910000,46.650002,46.349998,46.942000,-0.592002


In [389]:
forecast_df['percent_change'] = (forecast_df['diff']/forecast_df['average'])*100
forecast_df

,SYMBOL,URL,0,1,2,3,4,5,6,7,8,9,10,average,diff,percent_change
0,NKE,http://onlinenewsguru.com/2019/09/01/investors...,80.279999,81.129997,80.529999,82.739998,83.309998,80.440002,82.250000,82.029999,83.480003,85.379997,84.500000,82.156999,2.343001,2.851858
1,C,http://www.bnnbloomberg.ca/cp-rail-raises-quar...,69.669998,69.089996,69.199997,68.830002,68.730003,69.510002,71.029999,70.699997,69.959999,70.360001,70.669998,69.707999,0.961999,1.380040
2,UFPT,http://www.bnnbloomberg.ca/unilever-subsidiary...,35.270000,35.490002,35.740002,36.099998,37.400002,37.400002,36.150002,36.939999,36.810001,36.349998,36.240002,36.365001,-0.124999,-0.343733
3,TER,http://www.businesskorea.co.kr/news/articleVie...,30.780001,31.900000,32.169998,32.330002,31.490000,31.790001,31.709999,32.110001,32.900002,31.799999,31.930000,31.898000,0.032000,0.100319
4,EMMAW,http://www.globenewswire.com/news-release/2019...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,BYND,https://www.usatoday.com/story/money/2019/10/0...,142.990005,138.320007,154.339996,151.660004,148.619995,146.419998,143.300003,145.440002,145.720001,145.059998,142.729996,146.187001,-3.457005,-2.364783
2570,MSFT,https://www.wsj.com/articles/bright-horizons-t...,120.220001,117.050003,117.660004,117.910004,116.769997,116.930000,117.940002,119.019997,119.190002,119.970001,119.360001,118.266001,1.094000,0.925033
2571,AHH,https://www.wtrf.com/local/main-street-bank-an...,16.750000,16.700001,16.799999,16.790001,16.840000,16.940001,17.059999,17.020000,17.139999,17.240000,17.150000,16.928000,0.222000,1.311437
2572,TER,https://www.yahoo.com/entertainment/hgtvs-wind...,46.230000,45.450001,45.939999,46.650002,47.910000,48.830002,47.549999,47.299999,46.910000,46.650002,46.349998,46.942000,-0.592002,-1.261136


In [437]:
def label_sentiment(val, negative=-4.50, positive=4.20):
    if pd.isna(val): 
        return np.nan
    else: 
        if val <= negative: 
            return 'negative'
        elif val >= positive:
            return 'positive'
        else:
            return 'neutral'


forecast_df['sentiment'] = forecast_df['percent_change'].apply(lambda x: label_sentiment(x))
forecast_df

,SYMBOL,URL,0,1,2,3,4,5,6,7,8,9,10,average,diff,percent_change,sentiment
0,NKE,http://onlinenewsguru.com/2019/09/01/investors...,80.279999,81.129997,80.529999,82.739998,83.309998,80.440002,82.250000,82.029999,83.480003,85.379997,84.500000,82.156999,2.343001,2.851858,neutral
1,C,http://www.bnnbloomberg.ca/cp-rail-raises-quar...,69.669998,69.089996,69.199997,68.830002,68.730003,69.510002,71.029999,70.699997,69.959999,70.360001,70.669998,69.707999,0.961999,1.380040,neutral
2,UFPT,http://www.bnnbloomberg.ca/unilever-subsidiary...,35.270000,35.490002,35.740002,36.099998,37.400002,37.400002,36.150002,36.939999,36.810001,36.349998,36.240002,36.365001,-0.124999,-0.343733,neutral
3,TER,http://www.businesskorea.co.kr/news/articleVie...,30.780001,31.900000,32.169998,32.330002,31.490000,31.790001,31.709999,32.110001,32.900002,31.799999,31.930000,31.898000,0.032000,0.100319,neutral
4,EMMAW,http://www.globenewswire.com/news-release/2019...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,BYND,https://www.usatoday.com/story/money/2019/10/0...,142.990005,138.320007,154.339996,151.660004,148.619995,146.419998,143.300003,145.440002,145.720001,145.059998,142.729996,146.187001,-3.457005,-2.364783,neutral
2570,MSFT,https://www.wsj.com/articles/bright-horizons-t...,120.220001,117.050003,117.660004,117.910004,116.769997,116.930000,117.940002,119.019997,119.190002,119.970001,119.360001,118.266001,1.094000,0.925033,neutral
2571,AHH,https://www.wtrf.com/local/main-street-bank-an...,16.750000,16.700001,16.799999,16.790001,16.840000,16.940001,17.059999,17.020000,17.139999,17.240000,17.150000,16.928000,0.222000,1.311437,neutral
2572,TER,https://www.yahoo.com/entertainment/hgtvs-wind...,46.230000,45.450001,45.939999,46.650002,47.910000,48.830002,47.549999,47.299999,46.910000,46.650002,46.349998,46.942000,-0.592002,-1.261136,neutral


In [442]:
forecast_df[forecast_df['sentiment']=='neutral']

,SYMBOL,URL,0,1,2,3,4,5,6,7,8,9,10,average,diff,percent_change,sentiment
0,NKE,http://onlinenewsguru.com/2019/09/01/investors...,80.279999,81.129997,80.529999,82.739998,83.309998,80.440002,82.250000,82.029999,83.480003,85.379997,84.500000,82.156999,2.343001,2.851858,neutral
1,C,http://www.bnnbloomberg.ca/cp-rail-raises-quar...,69.669998,69.089996,69.199997,68.830002,68.730003,69.510002,71.029999,70.699997,69.959999,70.360001,70.669998,69.707999,0.961999,1.380040,neutral
2,UFPT,http://www.bnnbloomberg.ca/unilever-subsidiary...,35.270000,35.490002,35.740002,36.099998,37.400002,37.400002,36.150002,36.939999,36.810001,36.349998,36.240002,36.365001,-0.124999,-0.343733,neutral
3,TER,http://www.businesskorea.co.kr/news/articleVie...,30.780001,31.900000,32.169998,32.330002,31.490000,31.790001,31.709999,32.110001,32.900002,31.799999,31.930000,31.898000,0.032000,0.100319,neutral
5,CELG,http://www.globenewswire.com/news-release/2019...,93.279999,93.809998,93.589996,92.410004,91.930000,91.760002,91.529999,90.870003,90.910004,90.199997,89.470001,92.029000,-2.558999,-2.780644,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,BYND,https://www.usatoday.com/story/money/2019/10/0...,142.990005,138.320007,154.339996,151.660004,148.619995,146.419998,143.300003,145.440002,145.720001,145.059998,142.729996,146.187001,-3.457005,-2.364783,neutral
2570,MSFT,https://www.wsj.com/articles/bright-horizons-t...,120.220001,117.050003,117.660004,117.910004,116.769997,116.930000,117.940002,119.019997,119.190002,119.970001,119.360001,118.266001,1.094000,0.925033,neutral
2571,AHH,https://www.wtrf.com/local/main-street-bank-an...,16.750000,16.700001,16.799999,16.790001,16.840000,16.940001,17.059999,17.020000,17.139999,17.240000,17.150000,16.928000,0.222000,1.311437,neutral
2572,TER,https://www.yahoo.com/entertainment/hgtvs-wind...,46.230000,45.450001,45.939999,46.650002,47.910000,48.830002,47.549999,47.299999,46.910000,46.650002,46.349998,46.942000,-0.592002,-1.261136,neutral


In [30]:
forecast_df[forecast_df['average'].isnull()]

NameError: name 'forecast_df' is not defined

In [ ]:
forecast_df.to_csv("labeled_news.csv",  encoding='utf-8')

In [444]:
news_scraping_results_v2 = news_scraping_results.join(forecast_df[['sentiment']])

In [445]:
news_scraping_results_v2.to_csv('news_scraping_results.csv', encoding='utf-8')

In [40]:
news_scraping_results = news_scraping_results.drop(axis=1, columns='Unnamed: 0' )

In [43]:
news_scraping_results = pd.read_csv('./news_scraping_results.csv')


#news_scraping_results.to_csv('news_scraping_results.csv', encoding='utf-8')

In [44]:
news_scraping_results

,SYMBOL,SITE_NAME,URL,PUBLISH_TIME,SCRAPED_TIME,TITLE,CONTENT,sentiment
0,NKE,NaN,http://onlinenewsguru.com/2019/09/01/investors...,2019-09-01 07:00:00+00:00,2019-10-09-18-51,Investor's Roundup: Consolidated Water Co. Ltd...,george town cayman islands september 1 2019 sh...,neutral
1,C,NaN,http://www.bnnbloomberg.ca/cp-rail-raises-quar...,2019-05-06 21:55:22+00:00,2019-10-09-18-51,CP Rail raises quarterly dividend 27.5% after ...,cp rail raise quarterly dividend 27 5 after st...,neutral
2,UFPT,NaN,http://www.bnnbloomberg.ca/unilever-subsidiary...,2019-04-09 07:00:00+00:00,2019-10-09-18-52,Unilever subsidiary Schmidt's Natural to launc...,unilever plc un n be dip -pron- toe into the c...,neutral
3,TER,NaN,http://www.businesskorea.co.kr/news/articleVie...,2019-01-24 08:00:00+00:00,2019-10-09-18-51,Multinational Pharma Companies Often Stop Drug...,the korean operation of japanese multinational...,neutral
4,EMMAW,GlobeNewswire,http://www.globenewswire.com/news-release/2019...,2019-07-18 07:00:00+00:00,2019-10-08-18-04,Telemynd Provides Update Following Spinoff Tra...,mission viejo calif and torrance calif july 18...,NaN
...,...,...,...,...,...,...,...,...
2569,BYND,NaN,https://www.usatoday.com/story/money/2019/10/0...,2019-10-09 13:33:00+00:00,2019-10-09-18-51,Johnson & Johnson ordered to pay $8 billion to...,close oklahoma lawyer argue johnson johnson ag...,neutral
2570,MSFT,NaN,https://www.wsj.com/articles/bright-horizons-t...,2019-04-05 07:00:00+00:00,2019-10-09-18-51,"Bright Horizons Tests Parents’ Patience, Not N...",for many thing app be great for other -pron- m...,neutral
2571,AHH,NaN,https://www.wtrf.com/local/main-street-bank-an...,2019-06-20 07:00:00+00:00,2019-10-09-18-52,Main Street Bank anniversary and new announcem...,a local business be celebrate -pron- 18th anni...,neutral
2572,TER,NaN,https://www.yahoo.com/entertainment/hgtvs-wind...,2019-07-10 07:00:00+00:00,2019-10-09-18-50,HGTV's Windy City Rehab Stars Have Work Permit...,the future of hgtv show windy city rehab be un...,neutral


In [37]:
forecast_df.iloc[0]

NameError: name 'forecast_df' is not defined

In [42]:
news_scraping_results.to_csv('news_scraping_results.csv', index=False, encoding='utf-8')